In [5]:
!pip install pymongo
!pip install boto3

In [3]:
from pymongo import MongoClient
db = MongoClient("ec2-54-171-247-111.eu-west-1.compute.amazonaws.com:27017").inequality

In [78]:
import boto3
import pandas as pd
s3 = boto3.resource('s3')
client = boto3.client('s3')

def make_key(stat, part):
    file = stat.lower()+'_part_'+str(part)+'.csv'
    key = "01_inequality/04_Zonnal_Stats/"+stat+"/"+file
    return key

def get_csv(stat, part):
    client = boto3.client('s3')
    key = make_key(stat, part)
    obj = client.get_object(Bucket = "inequality-upf-cleaned-intersections", Key = key)
    return pd.read_csv(obj.get('Body'))

def get_obs(stat, part, obs):
    df = get_csv(stat, part)
    return df[df.obs == obs]

def get_parts(pop_part, pop_obs, nl_part, nl_obs):
    return pd.concat([get_obs("Pop", pop_part, pop_obs), get_obs("Night", nl_part, nl_obs)], axis=0)


In [83]:
get_obs("Pop", 143, 9)

,obs,pop2000_sum,pop2000_mean,pop2001_sum,pop2001_mean,pop2002_sum,pop2002_mean,pop2003_sum,pop2003_mean,pop2004_sum,...,pop2010_mean,pop2011_sum,pop2011_mean,pop2012_sum,pop2012_mean,pop2013_sum,pop2013_mean,pop2014_sum,pop2014_mean,part
109,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,143


In [275]:
def get_from_geo(g, fields, coll):
    return coll.find({ 'geometry': {'$geoWithin': { '$geometry': g }}}, 
                        fields)

def get_region(region, fields, coll = 'etl'):
    g = db.regions.find_one({'obs': region})['geometry']
    return get_from_geo(g, fields, db[coll])

def get_country(code, fields, coll = 'etl'):
    g = db.countries.find_one({'ADM0_CODE': code})['geometry']
    return get_from_geo(g, fields, db[coll])

In [282]:
import math

def cast(n):
    return 0 if (math.isnan(n) or math.isinf(n)) else n

def growth(y_t, y_prev):
    try:
        return (y_t**2)/float(y_prev) if y_t else 0 # 0 if y_t is 0
    except ZeroDivisionError:
        return y_t**2/.1 # infinite growth, how do we handle it????

def reducer(a, b):
    curr, prev = b
    running_growth, total = a
    return (running_growth + growth(cast(curr), cast(prev)), total + cast(curr))

def collector(l):
    return reduce(reducer, l, (0,0))

def average_growth(l):
    summed_growth, total = collector(l)
    try:
        return math.log(summed_growth / total)
    except ZeroDivisionError:
        return 0

In [290]:
import time

def calc_country_growth(country_code, name):
    cursor = get_country(country_code, {'gini13_F18': 1, 'gini12_F18': 1}) 
    ginis = ((d['gini13_F18'], d['gini12_F18']) for d in cursor)
    start = time.time()
    results = average_growth(ginis)
    end = time.time()
    print "GROWTH OF "+name+": "+str(results)+" calculated in "+str(end-start)+"ms"
    return results

In [291]:
puerto_rico = calc_country_growth(200, "Puerto Rico") # 0.75

GROWTH OF Puerto Rico: 0.206117015187 calculated in 17.7195289135ms


In [296]:
rwanda = calc_country_growth(205, "Rwanda") #2.05

GROWTH OF Rwanda: 0.530988487788 calculated in 78.9824380875ms


In [292]:
haiti = calc_country_growth(108, "Haiti") # 2.23

GROWTH OF Haiti: 0.309481163613 calculated in 84.6391928196ms


In [295]:
denmark = calc_country_growth(69, "Denmark") #4.39

GROWTH OF Denmark: 0.0436170105035 calculated in 157.00563097ms


In [293]:
switzerland = calc_country_growth(237, "Switzerland") #4.86

GROWTH OF Switzerland: 0.120637366927 calculated in 240.009339809ms


In [297]:
cuba = calc_country_growth(63, "Cuba") # 9.3

GROWTH OF Cuba: 0.316066194441 calculated in 328.703745842ms


In [294]:
north_korea = calc_country_growth(67, "North Korea") # 12.92

GROWTH OF North Korea: 0.561598206791 calculated in 571.456528902ms


In [ ]:
japan = calc_country_growth(126, "Japan") # 23.01